## ComCam Astrometry

Finding an a.net solution for one of the first images
Craig Lage - 25-Oct-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.summit.utils.plotting import plot
import lsst.afw.cameraGeom.utils as camGeomUtils

from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u
from astropy.table import Table
from astroquery.astrometry_net import AstrometryNet
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig

In [ ]:
butler = Butler('/repo/embargo_new', collections=["LSSTComCam/raw/all", "LSSTComCam/calib"])

## First, get a list of exposures
### These should match what you see in RubinTV.

In [ ]:
dayObs = 20241024
instrument = "LSSTComCam"

exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
                    where=f"exposure.day_obs={dayObs} and instrument='LSSTComCam'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter)


# Get the data from the headers

In [ ]:
expId = 2024102400089
mData = butler.get('raw.metadata', detector=4, exposure=expId, instrument=instrument)
for key in mData.keys():
    print(key, mData[key])

## Define a simple ISR
### Just overscan subtraction and bias subtraction.

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=True
isrConfig.usePtcGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

## Run the ISR and look at the result

In [ ]:
expId = 2024102400089
exp = butler.get('raw', detector=4, exposure=expId, instrument=instrument)
biasExp = butler.get('bias', detector=4, exposure=expId, instrument=instrument) # This is a bias image associated with the data
isrResult = isrTask.run(exp, bias=biasExp) # This runs the ISR

In [ ]:
%matplotlib inline
x = plot(isrResult.exposure, stretch='ccs')
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_{expId}.png")

# Plot a small region

In [ ]:
x = plot(isrResult.exposure.image.array[2225:2275, 1350:1400])

In [ ]:
charConfig = CharacterizeImageConfig()
charConfig.doMeasurePsf = True
charConfig.doApCorr = False
charConfig.doDeblend = False
charConfig.repair.doCosmicRay = False
charConfig.repair.doInterpolate = True   
charConfig.detection.minPixels = 500
charTask = CharacterizeImageTask(config=charConfig)


exp = isrResult.exposure
mData = exp.getMetadata()
charResult = charTask.run(exp)
sourceCatalog = charResult.sourceCat


In [ ]:
sources = sourceCatalog.asAstropy()
sources.keep_columns(['base_SdssCentroid_x', 'base_SdssCentroid_y', 'base_CircularApertureFlux_3_0_instFlux'])
sources.sort('base_CircularApertureFlux_3_0_instFlux', reverse=True)
len(sources)

In [ ]:
ast = AstrometryNet()
ast.api_key = 'xxawwhvleirxcswx'

In [ ]:
AstrometryNet.show_allowed_settings()

In [ ]:
image_width = 4072
image_height = 4000
scale_units = 'arcsecperpix'
scale_type='ev' # ev means submit estimate and % error
scale_est = 0.20
scale_err = 2.0
center_ra = mData['RA']
center_dec = mData['DEC']
radius = 0.5
wcs_header = ast.solve_from_source_list(sources['base_SdssCentroid_x'], sources['base_SdssCentroid_y'],
                                        image_width, image_height, scale_units=scale_units,
                                        scale_type=scale_type, scale_est=scale_est, scale_err=scale_err,
                                        center_ra=center_ra, center_dec=center_dec, radius=radius,
                                        crpix_center=True, solve_timeout=240)

In [ ]:
wcs_header

In [ ]:
print(f"Header center RA = {mData['RA']:.5f}, Anet center RA = {wcs_header['CRVAL1']:.5f}")
print(f"Header center Dec = {mData['DEC']:.5f}, Anet center Dec = {wcs_header['CRVAL2']:.5f}")

In [ ]:
RA_error = (wcs_header['CRVAL1'] - mData['RA']) * 3600.0
Dec_error = (wcs_header['CRVAL2'] - mData['DEC']) * 3600.0
print(f"RA error = {RA_error:.1f} arcsec, Dec error = {Dec_error:.1f} arcsec")

In [ ]:
x = plot(isrResult.exposure, stretch='ccs')
text1 = f"Header center RA = {mData['RA']:.5f}, Anet center RA = {wcs_header['CRVAL1']:.5f}"
text2 = f"Header center Dec = {mData['DEC']:.5f}, Anet center Dec = {wcs_header['CRVAL2']:.5f}"
RA_error = (wcs_header['CRVAL1'] - mData['RA']) * 3600.0
Dec_error = (wcs_header['CRVAL2'] - mData['DEC']) * 3600.0
text3 = f"RA error = {RA_error:.1f} arcsec, Dec error = {Dec_error:.1f} arcsec"
x.axes[0].text(800,1800, text1, color="cyan", fontsize=12, weight='bold')
x.axes[0].text(800,1600, text2, color="cyan", fontsize=12, weight='bold')
x.axes[0].text(800,1400, text3, color="cyan", fontsize=12, weight='bold')
x.axes[0].set_title(f"ComCam Astrometry {expId}", fontsize=24)
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Astrometry{expId}.png")